# 🧬 Geometry-Complete Equivariant Diffusion
## De Novo Drug Design Training

**Selection Strategy**:
- Size stratification (40 small + 40 medium + 40 large)
- Centroid clustering for diversity
- Stratified val split (preserve size ratios)
- Pocket-level split (no leakage)

**Result**: 100 train × 50 ligands = 5,000 pairs

## Cell 1: Setup

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import torch
print(f'GPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else "None!"}')
print(f'VRAM: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB')

!pip install -q torch-geometric rdkit scipy numpy pyyaml tqdm gdown scikit-learn
print('✅ Setup complete')

## Cell 2: Clone Repository

In [ ]:
import os
REPO = '/content/drive/MyDrive/geom_diffusion'
if not os.path.exists(REPO):
    !git clone https://github.com/Nethrananda21/geom_diffusion.git {REPO}
%cd {REPO}
!git pull origin master

## Cell 3: Download Pre-processed Data

In [ ]:
import os
import gdown

DATA_DIR = '/content/data/crossdocked'
os.makedirs(DATA_DIR, exist_ok=True)

files = {
    'train_data.pkl': '1vJyxCIqCYwP3qj4THMofdSd1rZDEQpPG',
    'val_data.pkl': '1FpVNcdj0R5YOsaLQm6T4D5QOKZGI4Xc5'
}

for fname, fid in files.items():
    path = f'{DATA_DIR}/{fname}'
    if not os.path.exists(path):
        print(f'📥 Downloading {fname}...')
        gdown.download(id=fid, output=path, quiet=False)
    else:
        print(f'✅ {fname} exists')

## Cell 4: Intelligent Pocket Selection

**Steps**:
1. Filter by size (≤250 atoms) and ligand count (≥50)
2. Stratify: small/medium/large bins
3. K-means diversity within each bin
4. **Stratified split**: Train/Val preserve size ratios

In [ ]:
import pickle
import numpy as np
from collections import defaultdict
from sklearn.cluster import KMeans

np.random.seed(42)

# Load data
with open(f'{DATA_DIR}/train_data.pkl', 'rb') as f:
    full_data = pickle.load(f)
print(f'Loaded {len(full_data)} samples')

# Group by pocket
pockets = defaultdict(list)
pocket_info = {}

for i, sample in enumerate(full_data):
    if hasattr(sample, 'keys'):
        pocket_id = sample.get('pocket_id', sample.get('receptor', f'pocket_{i}'))
        pocket_coords = np.array(sample.get('pocket_coords', sample.get('pocket_pos', [])))
    else:
        pocket_id = getattr(sample, 'pocket_id', f'pocket_{i}')
        pocket_coords = sample.pocket_pos.numpy() if hasattr(sample, 'pocket_pos') else np.zeros((1,3))
    
    pockets[pocket_id].append(sample)
    if pocket_id not in pocket_info:
        pocket_info[pocket_id] = {
            'size': len(pocket_coords),
            'centroid': pocket_coords.mean(axis=0) if len(pocket_coords) > 0 else np.zeros(3)
        }

print(f'Found {len(pockets)} unique pockets')

# === FILTER ===
MAX_SIZE, MIN_LIGANDS = 250, 50
valid = [p for p, s in pockets.items() 
         if pocket_info[p]['size'] <= MAX_SIZE and len(s) >= MIN_LIGANDS]
print(f'After filter: {len(valid)} pockets')

# === STRATIFY ===
small = [p for p in valid if pocket_info[p]['size'] <= 150]
medium = [p for p in valid if 150 < pocket_info[p]['size'] <= 200]
large = [p for p in valid if 200 < pocket_info[p]['size'] <= 250]

print(f'\nSize bins: Small={len(small)}, Medium={len(medium)}, Large={len(large)}')

# === K-MEANS DIVERSITY ===
def select_diverse(plist, n):
    if len(plist) <= n:
        return plist
    centroids = np.array([pocket_info[p]['centroid'] for p in plist])
    kmeans = KMeans(n_clusters=n, random_state=42, n_init=10)
    kmeans.fit(centroids)
    selected = []
    for c in range(n):
        cluster = [plist[i] for i in range(len(plist)) if kmeans.labels_[i] == c]
        if cluster:
            selected.append(cluster[0])
    return selected

# 40 from each bin = 120 total
sel_small = select_diverse(small, min(40, len(small)))
sel_medium = select_diverse(medium, min(40, len(medium)))
sel_large = select_diverse(large, min(40, len(large)))

print(f'Selected: {len(sel_small)} small + {len(sel_medium)} medium + {len(sel_large)} large')

# === STRATIFIED SPLIT (preserve ratios!) ===
# Train: 33 small, 33 medium, 34 large = 100
# Val: 7 small, 7 medium, 6 large = 20
np.random.shuffle(sel_small)
np.random.shuffle(sel_medium)
np.random.shuffle(sel_large)

train_small, val_small = sel_small[:33], sel_small[33:40]
train_medium, val_medium = sel_medium[:33], sel_medium[33:40]
train_large, val_large = sel_large[:34], sel_large[34:40]

train_pockets = train_small + train_medium + train_large
val_pockets = val_small + val_medium + val_large

print(f'\n✅ Train: {len(train_pockets)} pockets ({len(train_small)}S/{len(train_medium)}M/{len(train_large)}L)')
print(f'✅ Val: {len(val_pockets)} pockets ({len(val_small)}S/{len(val_medium)}M/{len(val_large)}L)')
print(f'✅ Overlap: {len(set(train_pockets) & set(val_pockets))} (must be 0!)')

# === CREATE DATASETS ===
LIGS_PER_POCKET = 50

train_samples = [s for p in train_pockets for s in pockets[p][:LIGS_PER_POCKET]]
val_samples = [s for p in val_pockets for s in pockets[p][:LIGS_PER_POCKET]]

print(f'\n📊 Train: {len(train_samples)}, Val: {len(val_samples)}')

# Save
with open(f'{DATA_DIR}/train_5k.pkl', 'wb') as f:
    pickle.dump(train_samples, f)
with open(f'{DATA_DIR}/val_1k.pkl', 'wb') as f:
    pickle.dump(val_samples, f)

# Save size info for stratified batching
train_sizes = {p: pocket_info[p]['size'] for p in train_pockets}
with open(f'{DATA_DIR}/train_pocket_sizes.pkl', 'wb') as f:
    pickle.dump(train_sizes, f)

print('💾 Saved: train_5k.pkl, val_1k.pkl, train_pocket_sizes.pkl')

## Cell 5: Test Large Pocket Batch (VRAM Safety)

In [ ]:
# Dry run: Ensure 4x large pockets don't OOM
import torch

large_samples = [s for p in train_large[:4] for s in pockets[p][:1]]  # 1 ligand each
max_pocket_size = max(pocket_info[p]['size'] for p in train_large[:4])

print(f'Testing 4 large pockets (max size: {max_pocket_size} atoms)')

# Estimate VRAM
# ~4 bytes per float32 * ~1000 features * batch_size * pocket_size
estimated_mb = (4 * 1000 * 4 * max_pocket_size) / 1e6
print(f'Estimated batch VRAM: ~{estimated_mb:.0f} MB')

if max_pocket_size <= 250:
    print('✅ Within T4 limits')
else:
    print('⚠️ May OOM - reduce pocket_atoms_max in config')

## Cell 6: Update Config

In [ ]:
import yaml

%cd /content/drive/MyDrive/geom_diffusion

with open('configs/debug_t4.yaml', 'r') as f:
    cfg = yaml.safe_load(f)

cfg['data']['root'] = '/content/data'
cfg['data']['train_file'] = 'crossdocked/train_5k.pkl'
cfg['data']['val_file'] = 'crossdocked/val_1k.pkl'
cfg['training']['max_epochs'] = 50
cfg['training']['batch_size'] = 4  # Safe for large pockets
cfg['hardware']['num_workers'] = 2

with open('configs/debug_t4.yaml', 'w') as f:
    yaml.dump(cfg, f)

print('✅ Config updated')

## Cell 7: Delete Cache

In [ ]:
import shutil
from pathlib import Path

for cache in ['/content/data/cache', 'data/cache']:
    if Path(cache).exists():
        shutil.rmtree(cache)
        print(f'🗑️ Deleted {cache}')

## Cell 8: Train 🚀

In [ ]:
%cd /content/drive/MyDrive/geom_diffusion
!python train.py --config configs/debug_t4.yaml --checkpoint_dir checkpoints

## Cell 9: Resume Training

In [ ]:
# After disconnect, run Cells 1, 2, 6, 7 then:
# %cd /content/drive/MyDrive/geom_diffusion
# !python train.py --config configs/debug_t4.yaml --resume checkpoints/best_model.pt